# Errata and Changes

+ __Feature Selection Pollution ⚠️__<br>
As in the [original exercise notebook](https://goo.gl/rQAj8S) ngram features for question 3 are still built in this revision from the entire dataset. This can be methodologically problematic; the test driver we use for scoring you submissions, provides only a train set portion of the data to your training function, which avoids this. 
<br><br>

+ __Input Data Cleanup ☢️__ <br>
In the original notebook provided, we had also provided and used fused words from the conllu data, in addition to using the decomposed forms of the same fused words. For example, due to the way that the Hebrew treebank was built, the data building function `build_verbs_data` would provide your code with both the word "והגישו" and the decomposition of that word into separated tokens "ו" and "הגישו". In this revision that is no longer the case. This means that the training data you get from this function is now also _smaller_, and has a little less tokens in the *non_verbs* category than before. Verify that your training algorithm still performs well, on this more correct makeup of input data.
<br><br>

+ __Ngrams Extraction__<br>
The ngram extraction code previously had a bug whereby not all ngrams would get extracted from every token being scanned due to wrongly put loop indexing. This had extremely little impact (and went undetected), as the data was large enough to capture most ngrams as is, however the current revision fixes that issue.
<br><br>

+ __Data Shape__<br>
In this revision, the shape of the return value of `build_verbs_data` has been modified for higher code safety; update your code accordingly. Basically we only changed its return value from a tuple to a dict. (but reach out to the [python forum](https://opal.openu.ac.il/mod/ouilforum/view.php?f=225720) in case you need help in adapting your code to this change).
<br><br>

+ __Few data validations added ...__
<br><br>

+ __Minor naming and readability improvements__
<br><br>

# Welcome

This notebook contains working code as the basis for your assignment. It fetches and loads the necessary data you will work on during this assignment, and demonstrates all the initial steps. Read through the notebook to confirm your understanding before proceeding to the assignment itself. ***The assignment questions for you to submit, appear right after the code portion of this notebook.***

# Classification | Verb Detection

## Assignment Overview


**In this assignment**, you will use two general purpose classification algorithms ― Naive Bayes and Logistic Regression ― to detect whether a token is a verb or not, in three different scenarios. Based on a dataset of tagged examples, you will leverage these algorithms to classify tokens yet-unseen during training, and work to improve the rate of success in this task. 

To do so, you will wrangle the art of ***feature engineering***: conceiving and implementing features that give the algorithm sufficient power to generalize from the seen examples, to previously unseen ones; it is typically, an iterative process until one finds good features that actually make that happen.

As part of the assignment, you will also self-study about few key ***evaluation metrics***, by which we measure the quality of a classifier. These metrics will gauge your progress as you work iteratively toward good scores (or at least, toward better ones than the baseline performance that this notebook, as published, demonstrates). 

## Prerequisite self-study


+ In order to complete this assignment, you should know what classification is. You are not supposed to learn the finest details of these specific classification algorithms, but reviewing the applicable book chapters on [Naive Bayes](https://web.stanford.edu/~jurafsky/slp3/4.pdf) and [Logistic Regression](https://web.stanford.edu/~jurafsky/slp3/5.pdf) can help; You will have taken, or would likely later take some course in Machine Learning, for a deep dive into those algorithms and comparable ones. In this course, we shall deep dive into NLP-specific algorithms during the forthcoming weeks, instead.


+ You should learn the meaning of the following evaluation metrics: precision, recall and accuracy, from this [online available book](https://web.stanford.edu/~jurafsky/slp3/4.pdf) (section 4.7 therein). 

## Technical Prerequisites
1. **Python 3.6.x. or higher** and Jupyter Notebook (for interacting with this notebook). 
<br>You may install python together with Jupyter, via the [Anaconda distribution](https://www.anaconda.com/distribution/#download-section), or in any other way that works for you. 
<br><br>
2. To successfully import a library in a notebook or python file, it must have been first _installed_ to your python environment, something that needs to be done outside of the notebook. Before running this notebook for the first time, run the following commands from the project's path:

   If you installed Juypter via Anaconda or Miniconda run the following library installation commands:
   ```
   conda install -c conda-forge --file requirements.txt 
   pip install pyconll==1.1.3
   ```

   If you installed Juypter via pip run the following library installation commands:
   ```
    pip install -r requirements.txt
    pip install pyconll==1.1.3
    ```

3. You should have [git](https://git-scm.com/) installed and working (i.e. the git command should be available on your system path) and an Internet connection. This is because the parts of this notebook that prepare the input datasets rely on git to fetch the input data sources.
<br><br>

## About <span style="color:skyblue">Universal Dependencies Treebanks</span>


In this assignment you will use *treebanks* as the input data. Let us first present what is a treebank:

+ Treebanks are a major linguistic resource for developing machine learning models.
+ A treebank is a corpus of sentences, annotated with their Syntactic Structures (hence the name "tree"). 
+ The treebanks we work with also marks the category that each token belongs to ― its Part of Speech (POS) e.g., a _verb_, an _adjective_, etc.
+ Typically, a treebank has been created through labour-intensive annotation by people trained and supervised about the specific annotation scheme being used. 


Specifically in this assignment, we will use treebanks that follow the modern [Universal Dependencies](https://universaldependencies.org/introduction.html) standard, so we will call them _Universal Dependencies Treebanks_.

In this assignment, we will merely use these treebanks in a simplified manner abstracting away most of their features, as you will learn much more about these concepts in later stages of this course.

## Getting Started with this notebook
1. Follow the cells and their outputs to confirm you understand the flow. <br>
2. To start off, it is advisable to make a local copy of the notebook ― <br>
making sure it is running for you with outputs similar to those frozen in time in this pre-run copy.
3. Then proceed with the assignments to submit, given right after the code portion of this notebook.

## <span style="color:gray">Imports</span>

In [1]:
## general imports
import random
import itertools 
from pprint import pprint  # slightly nicer printing of data structures
from tqdm import tqdm_notebook  # progress bar for loops (wraps an iterator for looping over it)
import pandas as pd  # tabular data manipulation library
from sklearn.model_selection import train_test_split  # data splitter

## imports for plotting
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from colab import enable_plotly_in_colab

## project supplied imports
import exercise_data_builder

## Downloading and loading the input data source
In this assignment we use a specific version of the Universal Dependencies Hebrew TreeBank as the input data source.
<br>⚙ make sure you have git installed and working on your OS before proceeding.

In [2]:
data = exercise_data_builder.build_verbs_data()
non_verbs, verbs, dual = data['non_verbs'], data['verbs'], data['dual']

transforming the data source...
skipped 37,035 fused surface forms


## Asserting the semantics of the data

In [3]:
# disjoint sets
for token, count in verbs.items():
    assert not token in non_verbs
    assert not token in dual
    
# fused tokens aren't included, only their decompositions
all_tokens = list(non_verbs.keys()) + list(verbs.keys()) + list(dual)
assert 'רגליים' in all_tokens and not 'וברגליים' in all_tokens

## Quick data exploration
Let us quantify some basic properties of the data, before diving into our tasks; This is typically called the Data Exploration phase.

In [4]:
print('{:,} tokens in the data are a non-verb'.format(len(non_verbs)))
print('{:,} tokens in the data are a verb'.format(len(verbs)))
print('{:,} tokens in the data are both verb and a non-verb'.format(len(dual)))

12,293 tokens in the data are a non-verb
5,014 tokens in the data are a verb
514 tokens in the data are both verb and a non-verb


In [5]:
# plotting boilerplate and then defining our plot
init_notebook_mode(connected=True)
enable_plotly_in_colab() 

fig = go.Figure(
        data=[go.Histogram(x=list(verbs.values()))], 
        layout=go.Layout(
            yaxis=dict(title='number of verbs having that frequency'), 
            xaxis=dict(title='frequency of verb occurence in the data')))

iplot(fig)

## Generating Features
After very basic exploration of the data, we approach generating features. We generate ngran features, and try using them for fitting a model that will be able to distinguish verbs from non-verbs.

### Character Ngrams
An _n-gram_ (or _ngram_) is defined as a unit of text of length $n$. 
<br> what is the unit of length? in our case we will be using character ngrams ― ngrams of $n$ characters each.

So 1-grams in our case will be strings of a single character, 2-grams will be strings of two characters and so on ....

Are character ngrams a good feature for training a classifier to distinguish verbs from non-verbs? let us see.

#### Extracting all ngrams (up to length `max_ngram_len`) from all the tokens we have in the corpus
Also, we count their unique occurences while at it

In [6]:
def add_to_ngrams(ngrams, max_ngram_len, token):
    
    ''' updates a pre-initialized ngrams dict with all ngrams present in the given input token '''
    
    # per ngram length
    for n in range(1, max_ngram_len + 1): 
        # sliding window iterate the token to extract its ngrams
        for idx in range(len(token) - n + 1):
            ngram = token[idx : idx + n] 

            if ngram in ngrams:
                ngrams[ngram] += 1
            else:
                ngrams[ngram] = 1    

    return ngrams # return value used for test only
            

assert add_to_ngrams(dict(), 2, "1234") == {'1': 1, '2': 1, '3': 1, '4': 1, '12': 1, '23': 1, '34': 1}
assert add_to_ngrams(dict(), 1, "1234") == {'1': 1, '2': 1, '3': 1, '4': 1}
assert add_to_ngrams(dict(), 0, "1234") == {}


In [7]:
ngrams = dict()

lexicon = list(verbs.keys()) + list(non_verbs.keys()) + list(dual) # all unique tokens in the data

max_ngram_len = 2

# for each token 
for token in tqdm_notebook(lexicon):  
    add_to_ngrams(ngrams, max_ngram_len, token)

#### The ngrams we got:

In [8]:
_1grams = list(filter(lambda ngram: len(ngram) == 1, ngrams))
_2grams = list(filter(lambda ngram: len(ngram) == 2, ngrams))

print('{} unique 1-grams extracted from the data'.format(len(_1grams)))
print('{} unique 2-grams extracted from the data'.format(len(_2grams)))
print('in total, {} unique ngrams extracted from the data'.format(len(ngrams)))

49 unique 1-grams extracted from the data
809 unique 2-grams extracted from the data
in total, 858 unique ngrams extracted from the data


In [9]:
for n in range(1, max_ngram_len+1):
    print("the unique {}-grams in the data:\n".format(n))
    print(sorted(filter(lambda ngram: len(ngram) == n, ngrams.keys())))
    print()

the unique 1-grams in the data:

['!', '"', '%', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '_', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת']

the unique 2-grams in the data:

['"א', '"ב', '"ג', '"ד', '"ה', '"ו', '"ח', '"ט', '"י', '"ך', '"כ', '"ל', '"ם', '"מ', '"ן', '"ס', '"ע', '"ף', '"ץ', '"ק', '"ר', '"ש', '"ת', ',0', ',1', ',2', ',3', ',5', ',6', ',7', ',8', ',9', '..', '.0', '.1', '.2', '.3', '.4', '.5', '.6', '.7', '.8', '.9', '.א', '.ב', '.ד', '.כ', '.ס', '.פ', '.ר', '0,', '0.', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '1,', '1.', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1פ', '2,', '2.', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '2ח', '2ש', '3,', '3.', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4,', '4.', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5,', '5

In [10]:
ngram_frequencies = pd.DataFrame.from_dict(ngrams, orient='index', columns=['occurences'])

print("ngram frequencies:")
with pd.option_context("display.max_rows", 10):
    display(ngram_frequencies)

ngram frequencies:


,occurences
א,3295
י,12374
ת,5941
ר,5844
אי,496
...,...
.ב,1
פם,1
זט,3
ש.,1


#### frequencies histogram

In [11]:
# some imports and setup for plotting
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from colab import enable_plotly_in_colab

init_notebook_mode(connected=True)
enable_plotly_in_colab()

data = [go.Histogram(x=ngram_frequencies['occurences'])]
fig = go.Figure(data=data, layout=go.Layout(
        yaxis=dict(title='number of ngrams having that frequency'), 
        xaxis=dict(title='ngram frequency in the data')))

iplot(fig)

## The Training Pipeline
In the following we experiment whether ngrams are enough for classifying tokens into verbs v.s. non-verbs. 
<br>This demonstrates the entire flow of classification.

In the first assignment questions that follow, you will need to improve this training pipeline by adding or removing features, or otherwise. So stay tuned to how the pipeline works, if anything is not clear, ask in the assignment's forum.

### Vectorization and Features

Vectorization is the turning of inputs into feature vectors; in our case, turning _tokens_ into feature vectors. It's the phase that turns text tokens, by nature symbolic entities, into numbers. Turning our tokens into numbers is what enables the use of machine learning algorithms like Naive Bayes and Logistic Regression, to operate on our data.

The aim of vectorization is to output a vector of numbers for every input that we throw at it. Every position of a feature vector will have a constant meaning: for example, we can define our feature vector to have the following semantics to its indices:

```
Position 0 ←→ the length of the input token
Position 1 ←→ whether the input token has the letter 'a'
Position 2 ←→ whether the last letter of the token is 'h'
```

And so on ....

The vecotrization function will take a token as input, and calculate the value for each of the features for that token. For example, using the above feature function, we'd get the following feature vectors for the following inputs:

```
angst → [5, 1, 0]
ouch → [4, 0, 1]
language → [8, 1, 0]

```

In [12]:
def vectorize(word):
    ''' our sample function for turning a given token into a feature vector '''
    
    # ngram occurences
    vec1 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if ngram in word:
            if vec1[idx]:
                vec1[idx] += 1
            else:
                vec1[idx] = 1
            
    # ngram occurences as prefix
    vec2 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if word.startswith(ngram):
            if vec2[idx]:
                vec2[idx] += 1
            else:
                vec2[idx] = 1


    # ngram occurences as suffix
    vec3 = [0] * len(ngrams)
    for idx, ngram in enumerate(ngrams):
        if word.endswith(ngram):
            if vec3[idx]:
                vec3[idx] += 1
            else:
                vec3[idx] = 1
        
    # our feature vector here is a concatenation of 3 feature types:
    return vec1 + vec2 + vec3 

A spec of our feature vector; This is a map mapping each index of our feature vectors to the meaning of the feature stored on that index. This is just a convenience, but it must be kept aligned to the vectorize function above in order to be valid. In an objected oriented style, we could have wrapped this spec and the vectorization function into a class.

In [13]:
feature_vector_description = \
    [dict(ngram=ngram, kind='ngram occurences in the input token for the ngram') for ngram in ngrams] + \
    [dict(ngram=ngram, kind='whether the input token starts with the ngram') for ngram in ngrams] + \
    [dict(ngram=ngram, kind='whether the input token ends with the ngram') for ngram in ngrams]

In [14]:
assert len(feature_vector_description) == len(vectorize('foo'))

this way, if we wanted, we can elegantly see the meaning of each position in our defined feature vector:

In [15]:
def feature_to_description(idx):
    return feature_vector_description[idx]

for feature_num in random.sample(range(len(feature_vector_description)), 10):
    feature_desc = feature_to_description(feature_num)
    print('for example, feature number {} represents {} {}'.format(
        feature_num, feature_desc['kind'], feature_desc['ngram']))
    

for example, feature number 28 represents ngram occurences in the input token for the ngram ס
for example, feature number 1169 represents whether the input token starts with the ngram יט
for example, feature number 295 represents ngram occurences in the input token for the ngram ער
for example, feature number 598 represents ngram occurences in the input token for the ngram 11
for example, feature number 618 represents ngram occurences in the input token for the ngram ח_
for example, feature number 550 represents ngram occurences in the input token for the ngram ף_
for example, feature number 1483 represents whether the input token starts with the ngram ת"
for example, feature number 2481 represents whether the input token ends with the ngram 3.
for example, feature number 2166 represents whether the input token ends with the ngram קק
for example, feature number 2044 represents whether the input token ends with the ngram לת


Taking our vectorization function for a spin over few inputs:

In [16]:
tokens = 'ברד ירד בדרום ספרד'.split()
    
for token in tokens:
    print('feature vector for the token {}:'.format(token))
    print(vectorize(token))
    print()
    

feature vector for the token ברד:
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Yes, the feature vectors we get with our current vectorization scheme are very sparse vectors!

**🛠 Software Engineering Note**: 
<br>in a mature implementation, we'd typically organize all vectorization functions in a dedicated python class or source file.

### Arranging the Data for Training 

So, we have our features defined in our vectorization function. We can now turn the data into the form that our machine learning algorithms expect:

1. We build a matrix $X$ where each line represents a token in our data. So the matrix $X$ will have $|t|$ rows, and $|f|$ columns, where $t$ is the number of tokens in our data and $f$ is the number of features we have defined in our vectorization function.
<br><br>
2. We also derive a _vector_ $y$ of the correct answer for each token; e.g., $0$ representing a _non-verb_, $1$ representing a _verb_ and $2$ representing a token that can serve as either of these roles.
<br><br>
3. We then use our data split function to split both $X$ and $y$ into a training and test split, so that we can train on the training split of the data, and evaluate on the test split.

<span style="color:green">Using the same data order for building $X$ and $y$ matters! the semantics are that the $nth$ row of $X$ has the class label of the $nth$ element of the vector $y$. That's how the training algorithms will use the data!</span><span style="color:gray"> So for example, if you were to change the order of elements in the vector $y$ by even a small way (or the order of rows in $X$), you'll be essentially feeding the model with totally wrong knowledge to learn from! 

In [17]:
## vectorizing the data
X_negative = list(map(vectorize, tqdm_notebook(non_verbs.keys(), desc = 'vectorizing non-verbs')))
X_positive = list(map(vectorize, tqdm_notebook(verbs.keys(), desc = 'vectorizing verbs')))
X_dual     = list(map(vectorize, tqdm_notebook(dual, desc = 'vectorizing duals')))

## building the given true labels vector in accordance
## we assign the value 0 to tokens that are not verbs, 1 to tokens that are verbs, and 2 to tokens that are ambiguous
y_negative = [0] * len(X_negative)
y_positive = [1] * len(X_positive)
y_dual     = [2] * len(X_dual)

X = X_negative + X_positive + X_dual
y = y_negative + y_positive + y_dual

assert len(X) == len(y)

So what do we have now? we built the feature matrix $X$ and a corresponding labels vector $Y$.
<br>We could look at them by converting them into pandas data types:

In [18]:
display(pd.DataFrame(X))

,0,1,2,3,4,5,6,7,8,9,...,2564,2565,2566,2567,2568,2569,2570,2571,2572,2573
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
display(pd.Series(y))

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
17791    2
17792    2
17793    2
17794    2
17795    2
17796    2
17797    2
17798    2
17799    2
17800    2
17801    2
17802    2
17803    2
17804    2
17805    2
17806    2
17807    2
17808    2
17809    2
17810    2
17811    2
17812    2
17813    2
17814    2
17815    2
17816    2
17817    2
17818    2
17819    2
17820    2
Length: 17821, dtype: int64

Now as promised, we split the data into a training set and a test set.
Each split comprises its own (X, y).

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

### Fitting the model
This step is the core of the model training pipeline!
<br>In this step, we feed the vectorized training data into the training algorithm of our choice.
In this task, we use one of the basic classification algorithm implementations of [scikit learn](https://scikit-learn.org/stable/), python's default machine learning library.

How does our vectorization affect the training time? we leave this to you to find out, but some tips:
+ At least with Naive Bayes, the more features and data we have, the longer this will take to complete.
+ The longer the feature vector, the more computation is taking place during training ...
+ 0 values in the feature vectors may incur less computational cost

How does training time trade-off with model performance? we leave the question open.

#### The classification algorithms for this assignment

Logistic Regression is commonly regarded the simplest _discriminative_ classification algorithm, whereas Naive Bayes is commonly regarded the simplest _generative_ classification algorithm. We will discuss generative v.s. discriminative in the next classes so don't worry about that for now. For the differences in performance, you may browse through [this seminal paper on this topic](https://ai.stanford.edu/~ang/papers/nips01-discriminativegenerative.pdf) published already in 2001, or in [follow-up papers](https://www.semanticscholar.org/paper/On-Discriminative-vs.-Generative-Classifiers%3A-A-of-Ng-Jordan/00bbbf3af78f80651e9f955209ee72711fa5d412).

Throughout this overall assignment, you are free to choose between these two algorithms, and only these two.

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# keep the following line uncommented to use Naive Bayes
model = MultinomialNB()

# keep the following line uncommented to use Logistic Regression
model = LogisticRegression(solver='liblinear', multi_class='auto')

Maybe we want to bias the classifier with sample weights over the training data, but for now this code applies uniform sample weights ...

In [22]:
sample_weights = []

for label in y_train:
    if label == 0:
        weight = 1
    elif label == 1:
        weight = 1
    elif label == 2:
        weight = 1
        
    sample_weights.append(weight)

Fitting the model:

In [23]:
model.fit(X_train, y_train, sample_weights); 
# the semicolon only avoids outputing the result of the last expression in the cell

#### Peeking into the trained model
Think about this as a white-box way of looking at the model after it's been "fitted" just above. 

In [24]:
model_coefficients = list(zip(model.coef_[0], feature_vector_description)) # zip merges the two lists, that's all
sorted(list(model_coefficients), reverse=True, key=lambda tuple: tuple[0])[:15]

[(2.506964977096644,
  {'ngram': '"', 'kind': 'ngram occurences in the input token for the ngram'}),
 (2.389162545561443,
  {'ngram': '_', 'kind': 'ngram occurences in the input token for the ngram'}),
 (2.255278339992075,
  {'ngram': '_', 'kind': 'whether the input token ends with the ngram'}),
 (2.173031780430532,
  {'ngram': 'קס',
   'kind': 'ngram occurences in the input token for the ngram'}),
 (2.0736170211277103,
  {'ngram': 'י', 'kind': 'whether the input token ends with the ngram'}),
 (2.0069523958964703,
  {'ngram': 'סט',
   'kind': 'ngram occurences in the input token for the ngram'}),
 (1.9169835458604967,
  {'ngram': 'ו', 'kind': 'whether the input token starts with the ngram'}),
 (1.6590237067247093,
  {'ngram': 'יה', 'kind': 'whether the input token ends with the ngram'}),
 (1.5282450579345634,
  {'ngram': 'צנ',
   'kind': 'ngram occurences in the input token for the ngram'}),
 (1.370620509114251,
  {'ngram': 'קא',
   'kind': 'ngram occurences in the input token for the 

## Model Evaluation

### So we have fitted the model now. How good is it?

In [25]:
words = ['הלבשה', 'התלבשו', 'התלבשה', 'הלבישה', 'התלבש', 'לבש', 'לבוש']
list(zip(words, (model.predict(list(map(vectorize, words))))))

[('הלבשה', 0),
 ('התלבשו', 1),
 ('התלבשה', 1),
 ('הלבישה', 0),
 ('התלבש', 1),
 ('לבש', 1),
 ('לבוש', 0)]

Looking at specific examples like above can be tempting, but it totally doesn't scale!
<br>To really evaluate our model, we will let it predict the labels of our test set!
<br><span style="color:gray">A note about terminology: for historical reasons, the act of giving a trained model a new sample and letting it output its class is called, _prediction_. Sometimes the term _inference_ is used instead of _prediction_.</span>

### Using the model to predict over the test set

#### Vectorizing and Arranging the Test Set
this step must always follow the same vector semantics as we used when preparing the training data for the training!

In [26]:
y_pred = model.predict(X_test) 

#### Per Token Accuracy
Accuracy is the ratio of tokens in the test set, for which the model's prediction was correct.
<br>We use the sklearn function here, but you could easily calculate this with a simple loop...

In [27]:
import sklearn.metrics 
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8171005385996409

#### A Confusion Matrix
A confusion matrix counts the classification errors by their kind. It shows how many times did the prediction for each class end up yielding a different class than the correct one. ([Refer to page 14 in this book chapter again](https://web.stanford.edu/~jurafsky/slp3/4.pdf) for further elucidation). 

Unfortunately, sklearn assumes we remember the class numbers rather than provide axis headers, so recall that the label $0$ had been earlier assigned in this notebook to the class _non-verb_, $1$ was assigned to the class _verb_ and $2$ the class _dual_. If you like, write your own code for printing a confusion matrix more nicely, as it only entails very simple counting ... here's sklearn's lean version of the confusion matrix for our prediction results over our test set:

In [28]:
 sklearn.metrics.confusion_matrix(y_test, y_pred)

array([[2812,  260,    2],
       [ 424,  829,    1],
       [  82,   46,    0]])

#### Per Class Precision, Recall and F1 Score
Precision and Recall conclude our tour of the most basic metrics for classification accuracy. We assume you have already studied about them in the [online available book](https://web.stanford.edu/~jurafsky/slp3/4.pdf) (section 4.7 therein) as you were asked to already above.

One popular way of providing intuition into Precision and Recall comes from the area of Information Retrieval: consider a radically simplified view of a search engine; imagine that every possible web page in the world is either _enitrely relevant_ or _entirely irrelevant_ to your search query. 

In that context, <span style="color:green">$Precision$</span> would be the ratio of results showing on the search results page that you actually find relevant for your search query, out of all search results showing; you would want Precision to therefore be high (as close as possible to $1$). <span style="color:green">$Recall$</span>, in that context, would be the ratio of useful results showing on the search results page, out of all relevant results out there. You would want Recall too, to be high and as close as possible to $1$. The F1 score is a fancy name for the harmonic average of the two, which is sometimes used for the simplicity of presentation. 

\* in practice, different training pipelines and classification algorithms will yield different trade-offs between Precision and Recall. Different trade-offs between Precision and Recall may be desireable in different settings, and in reality the concept of Area Under the Curve is used many times in practice, to compare the value of competing fitted models, an aspect we will not cover (nor require) in this exercise.

In [29]:
print(sklearn.metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88      3074
           1       0.73      0.66      0.69      1254
           2       0.00      0.00      0.00       128

   micro avg       0.82      0.82      0.82      4456
   macro avg       0.53      0.53      0.52      4456
weighted avg       0.79      0.82      0.80      4456



---

# Summarizing 

1. In the above, we have developed <span style="color:green">**a classifier**</span>, one you'll improve in this assignment.
2. It currently uses a basic set of features, based on character ngrams, to turn each token into a vector.
3. This classifier is based on library implementations of classification algorithms (Naive Bayes / Logistic Regression).
<br><br>
4. The <span style="color:green">**input**</span> to this classifier is a token 
5. The <span style="color:green">**output**</span> from this classifier is a class lablel:

   + $0$ for a token that is not a verb
   + $1$ for a token that is a verb
   + $2$ for a token that is dual (both a verb and a non-verb)
<br><br>   
6. We have trained on a training split of the data, and evaluated on a test split.
<br><br>
5. The classifier we've developed is not always right of course; 
<br>We measure its quality using several evaluation metrics that you have experienced above.


---

# The Assignment ⚑

## _3 pts_ | Question 1 

In order to understand how good or bad a model is, it is often good to compare to some relevant baseline. Assume you have no existing baseline to compare to, but you still wish to have _some_ point of reference to compare to. ***Suggest a naive and reasonable baseline for accuracy for the classification task implemented above, that does not rely on any machine learning or classification algorithm at all***. hint: think of the class imblanace in the data.

\* class imbalance is loosely defined as the extent to which one or more classes are much less (or more) frequent in the data relative to the other classes. it is no surprise then, that class imblanace can be directly seen in the training data; in terms of the classification report above, this corresponds to the term $Support$, which is a fancy name for the number of occurences of a class in the training data. 

Express your baseline as an accuracy score between 0 and 1.

## _2 pts_ | Question 2 

what can you say about the performance obtained above, according to your baseline?

## _22 pts_ | Question 3

Suggest and implement better classifier features helpful for the Hebrew language. implement the features and provide your implementation and the resulting performance report showing the performance improvement you were able to obtain. obviously, you will have to touch at least the vectorization part of this notebook to do so. work iteratively and analyze your attempts (experiments) to gain progress.

**Note:** <span style="color:#991111">the input spec and output spec for this task should be the same as implemented above, as summarized in the `Summary` section just above</span>. You should change the training process, via feature engineering and/or otherwise, to improve the evaluation metrics as best as you can.

## _22 pts_ | Question 4

now suggest and apply features specifically for English. Provide your implementation classifying over an English Universal Dependencies Treebank, https://github.com/UniversalDependencies/UD_English-EWT, including a performance report. Modify the training process, to get the best results that you can.

**Note:** The input spec and output spec for this task are the same as in question 3, except that you will use the English Treebank this time. 

*first, use the following code cell to obtain the course's chosen version of the English Treebank:*

In [30]:
data = exercise_data_builder.build_verbs_data("UD_English-EWT", "7be629932192bf1ceb35081fb29b8ecb0bd6d767")
non_verbs, verbs, dual = data['non_verbs'], data['verbs'], data['dual']

transforming the data source...


In [31]:
# asserting the semantics of the data
for token in verbs:
    assert not token in non_verbs
    assert not token in dual

## _44 pts_ | Question 5 

<span style="color:green">_In this question you will implement a classifier for an entirely different task._</span> You will no longer classify a word, context-less, but rather, you will be classifying words as part of the sentence they appear in. This will open the way for bringing in new features into play in your pipeline, and very hopefully, result in better bottom-line model performance. It will also remove the possibility for ambiguity, as our input data is annotated (someone has decided for us, in the Hebrew Treebank, which words appear in verb role in which do not). So, put differently, you will be classifying whether a token _in a given sentence_ is a verb or not.

The Hebrew Universal Dependencies Treebank will be the source of data for this exercise.

Importantly, let us first define the input and output spec for this classifier:

+ <span style="color:green">**Input**</span>: a sentence and a token to classify</span>
+ <span style="color:green">**Output**</span>: 
   + $1$ if the token serves as a verb in the input sentence
   + $0$ otherwise


**Notes:**
+ You may reuse the code of this notebook in any way, or write from scratch. 
  <br>Our general advise is: borrow from this notebook all and any code that you find useful, <br>rather than attempting to tweak the notebook in-place for this radically different task.
<br><br>
+ You will be graded based upon:

   + dilligence and creativity in adding reasonable features
   + the accuracy score on the test set, obtained when run for grading
   + reproducibility: does your submitted solution notebook actually run when downloaded for grading
   + cleanly structured, mildly self-explanatory or documented code


### <span style="color:green">Obtaining the Allowed Input Data for This Question
The input to your pipeline is automatically derived for you from the Hebrew Treebank courtesy of the following code. For this assingment you should train with the training set portion of it, and evaluate on the test portion. Take note of the Input/Output spec for your final classifier here, after the data sourcing and exploration cells that follow.

<span style="color:#BB5511">**Note:** you are not allowed to use any of the annotations of the original Treebank, other than the verb/non-verb labels that our data sourcing code carries over from the original Treebank!</span>
    
Here is the training set and the test set for this task:

In [32]:
train_set, test_set = exercise_data_builder.build_verb_in_context_data()

transforming the data source...


Below there's one sample entry from this input (the first entry). 
<br>we could have printed it as is, but we use a pandas DataFrame for convenience of display. <br>As you see, the sentences have been both morphologically disambiguated already, and also partially morphologically decomposed!

In [33]:
pd.DataFrame.from_dict(train_set[100])

,label,token
0,0,דווקא
1,0,כיוון
2,0,ש
3,0,הוא
4,1,מתבונן
5,0,מוכשר
6,0,כל
7,0,-
8,0,כך
9,0,ה


So the values of the array `sentences` here, are the input data for training your classifier. No additional input data is allowed.

### Some initial data exploration over this data

#### counting the number of verbs per sentence

In [34]:
verb_counts = []
for entry in train_set:
    verbs = 0
    for token in entry:
        if token['label'] == 1:
            verbs += 1 
    
    verb_counts.append(verbs)

looking at raw data can be overwhelming:

In [35]:
sorted(verb_counts)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


So it is helpful to use a more concise view of the counts data:
we exemplify here the use of the [pandas library](https://towardsdatascience.com/pandas-series-a-lightweight-intro-b7963a0d62a2). pandas is _the python library_ for series and tabular data exploration and manipulation. you are free to use it for that. specifically what we do here, is bin the counts: instead of looking at how many verbs did every sentece have, we will look at how many sentences we have with 0 verbs, with 1 verbs, with 2 verbs, etc... this will be a more concise view of the data but still give us a sense of what's going on in the data.

In [36]:
# importing pandas 
import pandas as pd 

# turning the array we got, into a pandas series
verb_counts = pd.Series(verb_counts)

# binning the array values 
hist = verb_counts.value_counts().sort_index().to_frame('sentences')
hist.index.name = 'verbs'
hist

,sentences
verbs,
0,412
1,1391
2,1471
3,927
4,532
5,282
6,123
7,57
8,22


In [37]:
# notebook plot boilerplate
init_notebook_mode(connected=True)
enable_plotly_in_colab() 

# our plot
fig = go.Figure(
        data = [go.Bar(x=hist.index, y=hist['sentences'])], 
        layout = go.Layout(
            yaxis=dict(title='sentences'), 
            xaxis=dict(title='verbs')))

iplot(fig)

So, we've shown some basic data exploration of this dataset, and your input for training in this assignment is now in the variable `train_set`, of which we show 10 random elements below.

In [38]:
random.sample(train_set, 10)

[[{'token': 'ב', 'label': 0},
  {'token': 'יום', 'label': 0},
  {'token': 'חופשה_', 'label': 0},
  {'token': '_של_', 'label': 0},
  {'token': '_הם', 'label': 0},
  {'token': 'ה', 'label': 0},
  {'token': 'שבועי', 'label': 0},
  {'token': 'הם', 'label': 0},
  {'token': 'מגיעים', 'label': 1},
  {'token': 'ל', 'label': 0},
  {'token': 'ה_', 'label': 0},
  {'token': 'ספרייה', 'label': 0},
  {'token': 'ה', 'label': 0},
  {'token': 'פולנית', 'label': 0},
  {'token': 'כדי', 'label': 0},
  {'token': 'להציץ', 'label': 1},
  {'token': 'ב', 'label': 0},
  {'token': 'ה_', 'label': 0},
  {'token': 'עיתון', 'label': 0},
  {'token': ',', 'label': 0},
  {'token': 'ה', 'label': 0},
  {'token': 'מזכיר', 'label': 1},
  {'token': 'את', 'label': 0},
  {'token': 'ה', 'label': 0},
  {'token': 'בית', 'label': 0},
  {'token': '.', 'label': 0}],
 [{'token': 'ה', 'label': 0},
  {'token': 'סיפור', 'label': 0},
  {'token': 'גם', 'label': 0},
  {'token': 'דרש', 'label': 1},
  {'token': 'מאמץ', 'label': 0},
  {'toke

### <span style="color:green">Input Spec for this Classifier</span>

The input to the classifier should be a sentence with an index (of a token) in the sentence to predict

### <span style="color:green">Output Spec for this Classifier</span>

The output from your classifier should be $0$ for a token that is a verb in the input sentence, and $1$ for a token that is not.

## _3 pts_ | Question 6

Explain in a short paragraph the flow that takes place inside a Naive Bayes classifier, when it predicts the class of an input.

## _4 pts_ | Question 7

Explain in a short paragraph the flow that takes place inside a Logistic Regression classifier, when it predicts the class of an input.

## _20 bonus pts_ | Question 8 | Optional

*Manual domain adaptation.* 
<br>
1. obtain a small unrelated Hebrew dataset; 

    + this can be from a liberated copy of your gmail inbox ([google data takeout](https://takeout.google.com/settings/takeout))
    + sentences from a book that you have
    + from a whatsapp data export
    + from public hebrew tweets
    + from the hebrew wikipedia


2. Extract 30-50 sentences from that dataset, and annotate them (verb v.s. non-verb)
<br><br>
3. Use your best accomplished model from question 5 to predict them
<br><br>

<span style="color:skyblue">
Submit for this question:
1. <span style="color:skyblue">The new sentences with your annotation 
2. <span style="color:skyblue">The evaluation metrics you got for them
3. <span style="color:skyblue">An initial error analysis:
   1. how much did performance deteriorate compared to the performance on the original Hebrew Treebank?
   2. a brief analysis of the error cases that you detect.


<br><br>
<span style="color:gray">what not to include in your submission for this question?</span>
+ <span style="color:gray">You may not include in your new data, any data explicitly or implicitly disclosing the identity or personal information of any person/s. Anonymize any non-public data that you use, as necessary to comply with this requirement; you may therefore sanitize your data manually if you like.</span>
+ <span style="color:gray">You may not include any proprietary or confidential data in your new data.</span>

## _20 bonus pts_ | Question 9 | Optional

Implement either the _Multinomial Naive Bayes_ algorithm or the _Logistic Regression_ algorithm from scratch (using the numpy library for math is allowed). Establish the same performances as obtained with the ready-made sklearn implementation that you have used so far, or as close to that as possible.

What to submit for this?
+ a working notebook using your own implemented classifier rather than the sklearn one, per each of the programming assignments you have already implemented above


## <span style="color:orange">Submission Notes</span>

+ <span style="color:orange">Any question submission that does not actually run (halts with an error, or times out) will immediately deduce 5 pts for the question at hand. Any re-submission that does not actually run (halts with an error) will deduce up to additional 4 pts each. So properly make sure to verify that your code submission runs before submitting.</span>
+ <span style="color:orange">non-reproducible results will be ignored.</span>
+ <span style="color:gray">if any of your code should take more than 10 minutes to run or require more than 16GB of memory, please seek approval from the course staff in advance via email: openu.nlp.course@gmail.com.

